In [1]:
import pandas as pd
import numpy as np

In [2]:
def correct_countrycode(countrycode):
    '''
    Corrects countrycodes in the database that don't correspond to official 3 letters codes.
    '''
    if countrycode=='TMP':
        countrycode='TLS'
    if countrycode=='ZAR':
        countrycode='COD'
    if countrycode=='ROM':
        countrycode='ROU'
    return countrycode

In [8]:
data1 = pd.read_csv("data/disasters/gar_for_model.csv")

In [9]:
data1.columns

Index(['Unnamed: 0', 'country_name', 'Income group',
       'EXPOSED VALUE NATIONAL million US$', 'EARTHQUAKE million US$',
       'EARTHQUAKE per th', 'TROPICAL CYCLONES million US$',
       'TROPICAL CYCLONES per th', 'TSUNAMI million US$', 'TSUNAMI per th',
       'FLOODS million US$', 'FLOODS per th', 'MULTI-HAZARD million US$',
       'MULTI-HAZARD per th', 'ISO', 'wbregion', 'wbregionname', 'wbincome',
       'wbincomename', 'wblending', 'wbhipc'],
      dtype='object')

In [15]:
codes = pd.read_csv('wbccodes2014.csv')

In [16]:
codes.rename(columns={'country':'iso3'},inplace=True)

In [18]:
codes['iso3'] = codes.iso3.apply(correct_countrycode)

In [22]:
data1['iso3'] = data1.country_name.replace(codes.set_index("country_name").iso3)

In [12]:
df = pd.read_csv("data/disasters/df_original.csv")

In [13]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'country', 'gdp_pc_pp', 'pop', 'share1',
       'plgp', 'unemp', 'axfin_p', 'axfin_r', 'bashs', 'ophe', 'social_p',
       'social_r', 'axhealth', 'pov_head', 'shew', 'prepare_scaleup',
       'finance_pre', 'rating', 'avg_prod_k', 'protection', 'fa', 'pe', 'v',
       'pv', 'v_s', 'peref', 'faref', 'protectionref', 'share1_ref', 'H',
       'alpha', 'T_rebuild_K', 'T_rebuild_L', 'pi', 'income_elast',
       'original_country', 'iso3'],
      dtype='object')

In [23]:
aal = data1.reset_index().merge(df[['iso3','v','avg_prod_k']],on='iso3').set_index('iso3')

In [24]:
aal["floods"] = aal["FLOODS per th"]/10/aal.avg_prod_k/aal.v

In [25]:
aal["cyclones"] = aal["TROPICAL CYCLONES per th"]/10/aal.avg_prod_k/aal.v

In [26]:
aal.to_csv("data/disasters/aal_from_gar.csv")

In [3]:
drought_data = pd.DataFrame()

In [4]:
for model in ['GFDL','hadGEM','IPSL','MIROC','NorESM']:
    datatemp = pd.read_excel("data/disasters/droughts_data.xlsx",\
                             sheetname='ISI_{}_uncorr'.format(model))
    datakeep = datatemp[['ISO','RCP85_2030_SSP4_RP100','RCP85_2030_SSP5_RP100','hist_RP100']].set_index('ISO')
    datakeep.rename(columns={'RCP85_2030_SSP4_RP100':'RCP85_2030_SSP4_RP100_{}'.format(model),\
                             'RCP85_2030_SSP5_RP100':'RCP85_2030_SSP5_RP100_{}'.format(model),\
                             'hist_RP100':'hist_RP100_{}'.format(model)},\
                   inplace=True)
    drought_data['RCP85_2030_SSP4_RP100_{}'.format(model)] = datakeep['RCP85_2030_SSP4_RP100_{}'.format(model)]-\
                                                             datakeep['hist_RP100_{}'.format(model)]
    drought_data['RCP85_2030_SSP5_RP100_{}'.format(model)] = datakeep['RCP85_2030_SSP5_RP100_{}'.format(model)]-\
                                                             datakeep['hist_RP100_{}'.format(model)]

In [5]:
drought_data['max_SSP4'] = drought_data[['RCP85_2030_SSP4_RP100_{}'.format(model) for model in \
              ['GFDL','hadGEM','IPSL','MIROC','NorESM']]].max(axis=1)
drought_data['max_SSP5'] = drought_data[['RCP85_2030_SSP5_RP100_{}'.format(model) for model in \
              ['GFDL','hadGEM','IPSL','MIROC','NorESM']]].max(axis=1)
drought_data['min_SSP4'] = drought_data[['RCP85_2030_SSP4_RP100_{}'.format(model) for model in \
              ['GFDL','hadGEM','IPSL','MIROC','NorESM']]].min(axis=1)
drought_data['min_SSP5'] = drought_data[['RCP85_2030_SSP5_RP100_{}'.format(model) for model in \
              ['GFDL','hadGEM','IPSL','MIROC','NorESM']]].min(axis=1)

In [10]:
temp = drought_data.reset_index().merge(codes[['ISO','wbregion']],on='ISO')

In [11]:
ssp = pd.read_csv("ssp_data/SspDb_country_data_2013-06-12.csv")

In [12]:
ssp.UNIT.unique()

array(['million', 'billion US$2005/yr', '%'], dtype=object)

In [13]:
def get_pop_data_from_ssp(ssp_data,ssp,year):
    model='IIASA-WiC POP'
    if ssp==4: 
        ssptemp='4d'
    else:
        ssptemp=ssp
    scenario="SSP{}_v9_130115".format(ssptemp)
    selection=(ssp_data['MODEL']==model)&(ssp_data['SCENARIO']==scenario)
    pop_tot=ssp_data.ix[selection&(ssp_data['VARIABLE']=="Population"),[str(year),'REGION']]
    pop_tot[str(year)] = 10**(6)*pop_tot[str(year)]
    return pop_tot.rename(columns={'REGION':'ISO',str(year):'tot_pop_ssp{}'.format(ssp)})

In [14]:
ssp4_pop = get_pop_data_from_ssp(ssp,4,2030)
ssp5_pop = get_pop_data_from_ssp(ssp,5,2030)

In [15]:
ssp4_pop.head()

,tot_pop_ssp4,ISO
155875,414556.989,BHS
156070,353660.779,GUF
156265,103290.808,GRD
156460,3566497.411,BIH
156655,343934.049,MYT


In [16]:
temp = temp.merge(ssp4_pop,on='ISO')

In [17]:
temp = temp.merge(ssp5_pop,on='ISO')

In [18]:
temp ['max_SSP4'] = temp ['max_SSP4']/temp['tot_pop_ssp4']
temp ['max_SSP5'] = temp ['max_SSP5']/temp['tot_pop_ssp5']
temp ['min_SSP4'] = temp ['min_SSP4']/temp['tot_pop_ssp4']
temp ['min_SSP5'] = temp ['min_SSP5']/temp['tot_pop_ssp5']

In [19]:
test = temp.dropna().groupby('wbregion').mean()

In [20]:
temp ['max_SSP4_region'] = temp.wbregion.replace(test.max_SSP4)
temp ['max_SSP5_region'] = temp.wbregion.replace(test.max_SSP5)
temp ['min_SSP4_region'] = temp.wbregion.replace(test.min_SSP4)
temp ['min_SSP5_region'] = temp.wbregion.replace(test.min_SSP5)

In [21]:
temp.set_index('ISO').to_csv("data/disasters/drought_deltares.csv")

In [22]:
hist_drought = pd.read_excel("data/disasters/droughts_data.xlsx",sheetname='EUWATCH_ENSEMBLEMEAN_FUTURE')
hist_drought = hist_drought[['ISO','HIST_RP100']]
hist_drought = hist_drought.merge(codes[['ISO','wbregion']],on='ISO')
hist_drought.set_index('ISO').to_csv("data/disasters/hist_drought_deltares.csv")

In [118]:
np.isnan(temp.set_index('ISO').loc['ALB','RCP85_2030_SSP4_RP100_GFDL'])

True

In [119]:
coastal = pd.read_excel("data/disasters/costal_floods_data.xlsx")

In [124]:
coastal = coastal.merge(codes.rename(columns={'country_name':'Name'})[['ISO','Name']],on='Name')

In [125]:
pop_2010 = get_pop_data_from_ssp(ssp,5,2010)

In [127]:
coastal = coastal.merge(pop_2010,on='ISO')

In [128]:
coastal.columns

Index(['Name', 'Region', 'exposed_population_coastal_flooding_2010', 'ISO',
       'tot_pop_ssp5'],
      dtype='object')

In [129]:
coastal['share_2010'] = coastal.exposed_population_coastal_flooding_2010/coastal.tot_pop_ssp5

In [130]:
coastal[['share_2010','ISO']].set_index('ISO').to_csv('data/disasters/costal_floods_deltares.csv')